## Reading the DHS survey and GPS datasets

Importing necessary libraries

In [1]:
import geopandas as gpd

Reading the shapefile to understand cluster locations

In [4]:
shapefile = gpd.read_file("../poverty_predictor/data/GPS/RWGE61FL.shp")

In [5]:
shapefile.head()

,DHSID,DHSCC,DHSYEAR,DHSCLUST,CCFIPS,ADM1FIPS,ADM1FIPSNA,ADM1SALBNA,ADM1SALBCO,ADM1DHS,...,DHSREGCO,DHSREGNA,SOURCE,URBAN_RURA,LATNUM,LONGNUM,ALT_GPS,ALT_DEM,DATUM,geometry
0,RW201000000001,RW,2010.0,1.0,RW,RW15,Southern,NULL,NULL,2.0,...,7.0,Huye,GPS,R,-2.532818,29.684726,1706.0,1702.0,WGS84,POINT (29.68473 -2.53282)
1,RW201000000002,RW,2010.0,2.0,RW,RW11,Eastern,NULL,NULL,5.0,...,26.0,Gatsibo,GPS,R,-1.833858,30.310689,1631.0,1631.0,WGS84,POINT (30.31069 -1.83386)
2,RW201000000003,RW,2010.0,3.0,RW,RW14,Western,NULL,NULL,3.0,...,16.0,Ngororero,GPS,R,-1.888155,29.478298,2310.0,2324.0,WGS84,POINT (29.47830 -1.88816)
3,RW201000000004,RW,2010.0,4.0,RW,RW11,Eastern,NULL,NULL,5.0,...,28.0,Kirehe,GPS,R,-2.366763,30.521692,1388.0,1399.0,WGS84,POINT (30.52169 -2.36676)
4,RW201000000005,RW,2010.0,5.0,RW,RW11,Eastern,NULL,NULL,5.0,...,30.0,Bugesera,GPS,R,-2.171266,30.018541,1437.0,1445.0,WGS84,POINT (30.01854 -2.17127)


Reading the .dat file to understand the wealth asset data

Using the above shapefile to get average wealth of each cluster